In [1]:
%reset -f
import h5py,os,time
import numpy as np
import numpy.matlib
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve
from scipy.linalg import block_diag

#set physical constants
echarge=1.602e-19 #elementary charge [C]
perm0=8.85e-12 #vacuum permittivity [F/m]
kA=6.022e23 #Avogadro constant [1/mol]
kB=1.381e-23 #Boltzmann's constant [J/K]

def set_globvars(infile):
    global nodes,elements,faces,midpoints #mesh variables
    global node_flags,zones,face_flags #variables for performing tests
    global mask_e,mask_d,mask_s #boolean element/node/face indicies for volume/S1/S2
    global cx,cy,cz #PDE coefficients for scalar field
    global alphax,alphay,alphaz #PDE coefficients for scalar field
    global betax,betay,betaz #PDE coefficients for scalar field
    global gammax,gammay,gammaz #PDE coefficients for scalar field
    global a,f,g,q,s #PDE coefficients for scalar field
    global cx33,cy33,cz33 #PDE coefficients for vector field
    global alphax33,alphay33,alphaz33 #PDE coefficients for vector field
    global betax33,betay33,betaz33 #PDE coefficients for vector field
    global gammax33,gammay33,gammaz33 #PDE coefficients for vector field
    global a33,f3,g3,q33,s3 #PDE coefficients for vector field
    global rel_perm,tK,cinf,zval #physical properties
    global kd #scaling coefficients for distance
    
    #load pde coefficients
    print('Loading %s'%infile)
    print('')
    data=np.load(infile)
    nodes=data['nodes']
    elements=data['elements']
    faces=data['faces']
    
    node_flags=data['node_flags']
    zones=data['zones']
    face_flags=data['face_flags']
    
    mask_e=data['mask_e'] #computational element indices
    mask_d=data['mask_d'] #node indicies on Dirichlet boundary (S1)
    mask_s=data['mask_s'] #surface element indicies on the boundary of the third kind (S2)
    
    cx=data['cx']
    cy=data['cy']
    cz=data['cz']
    alphax=data['alphax']
    alphay=data['alphay']
    alphaz=data['alphaz']
    betax=data['betax']
    betay=data['betay']
    betaz=data['betaz']
    gammax=data['gammax']
    gammay=data['gammay']
    gammaz=data['gammaz']
    a=data['a']
    f=data['f']
    g=data['g']
    q=data['q']
    s=data['s']
    
    cx33=data['cx33']
    cy33=data['cy33']
    cz33=data['cz33']
    alphax33=data['alphax33']
    alphay33=data['alphay33']
    alphaz33=data['alphaz33']
    betax33=data['betax33']
    betay33=data['betay33']
    betaz33=data['betaz33']
    gammax33=data['gammax33']
    gammay33=data['gammay33']
    gammaz33=data['gammaz33']
    a33=data['a33']
    f3=data['f3']
    g3=data['g3']
    q33=data['q33']
    s3=data['s3']
    
    rel_perm=data['rel_perm']
    tK=data['tK']
    cinf=data['cinf']
    zval=data['zval']
    
    #compute middle point of each elements (efficiency to be improved)
    nelem=len(elements[:,0])
    midpoints=np.zeros((nelem,3))
    for i in range(0,nelem):
        for j in range(0,3):
            midpoints[i,j]=.25*sum(nodes[elements[i,:],j])
    
    nind_e=np.unique(elements[mask_e].flatten(order='C'))
    print('THE NUMBER OF COMPUTATIONAL NODES IS:',len(nind_e))
    print('THE NUMBER OF COMPUTATIONAL ELEMENTS IS:',np.count_nonzero(mask_e))
    print('THE NUMBER OF NODES ON S1 IS:',np.count_nonzero(mask_d))
    print('THE NUMBER OF ELEMENTS ON S2 IS:',np.count_nonzero(mask_s))
    print('')
    
    print('Scale the PDE equation for distance')
    debye_len=np.sqrt(rel_perm*perm0*kB*tK/2/zval**2/echarge**2/kA/cinf)
    kd=10**(-round(np.log10(debye_len))) #to be determined from debye length
    nodes=nodes*kd
    print('Distance scaling coefficient is: %.2e'%kd)
    print('')
    
    alphax=alphax/kd
    alphay=alphay/kd
    alphaz=alphaz/kd
    
    betax=betax/kd
    betay=betay/kd
    betaz=betaz/kd
    
    gammax=gammax/kd
    gammay=gammay/kd
    gammaz=gammaz/kd
    
    #make sure a,f,g,q were scaled by perm0 before
    #actually all PDE coefficients should be scaled by perm0
    #scale everything in main.py in later modifications
    #by adding an option scale_perm0 or not in build_pde.py
    #then call scale_pde function in main.py after set_globvars
    #unscale perm0 for now
    a=a/kd/kd
    f=f/kd/kd
    
    g=g/kd
    q=q/kd #not really tested this q
    
    #scale PDE coefficients for vector unknowns
    alphax33=alphax33/kd
    alphay33=alphay33/kd
    alphaz33=alphaz33/kd
    
    betax33=betax33/kd
    betay33=betay33/kd
    betaz33=betaz33/kd
    
    gammax33=gammax33/kd
    gammay33=gammay33/kd
    gammaz33=gammaz33/kd
    
    a33=a33/kd/kd
    f3=f3/kd/kd
    g3=g3/kd
    q33=q33/kd
    
    return
